In [ ]:

from requests import session
import matplotlib.pyplot as plt
import pandas as pd
import json
import copy
import numpy as np, scipy.stats as st
import json
from tqdm import tqdm
import pickle
import logging
import  argparse
import math
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from itertools import chain
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
logging.basicConfig(level=logging.INFO)
from sklearn import preprocessing
import matplotlib
from re import S
from scipy.stats.stats import pearsonr
from sklearn.calibration import calibration_curve
import matplotlib
from sklearn.metrics import roc_auc_score
import sys
import pickle
import os
# sys import ..
sys.path.append("../")
from data_split import *
from cdhf import CDHF
from tqdm import tqdm
import datetime
from scipy import stats
date_now = datetime.datetime.now()
date_now = date_now.strftime("%Y-%m-%d_%H%M%S")


In [ ]:
# NOTE: THIS FILE MUST BE CREATED BEFORE RUNNING THIS SCRIPT
with open('m1_m2_probs.pkl', 'rb') as f:
        data = pickle.load(f)

labels_test = data['labels_test']
m1_probs_test = data['m1_probs_test']
m2_probs_test = data['m2_probs_test']
labels_val = data['labels_val']
m1_probs_val = data['m1_probs_val']
m2_probs_val = data['m2_probs_val']

# for all probs, do 1 -
m1_probs_test = 1 - m1_probs_test
m2_probs_test = 1 - m2_probs_test
m1_probs_val = 1 - m1_probs_val
m2_probs_val = 1 - m2_probs_val



In [ ]:
cond_acc_targets = np.arange(79, 101,2)
m1_use_targets = np.array([0,0.1,0.25,0.5,0.75,0.9,1])
ours_acc = []
ours_hidden = []
ours_m1use = []

for m1_use in m1_use_targets:

    ours_acc_trial =[]
    ours_hidden_trial = []
    ours_m1use_trial = []

    for cond_acc_cst in cond_acc_targets:
        model_ours = CDHF(cond_acc_cst/100, 0.0, 0.1,100, m1_use)
        model_ours.fit(m1_probs_val, m2_probs_val, labels_val)
        metrics_test = model_ours.get_metrics(m1_probs_test, m2_probs_test, labels_test)
        ours_hidden_trial.append(metrics_test['often hidden'])
        ours_acc_trial.append(metrics_test['cond accuracy'])
        ours_m1use_trial.append(metrics_test['coverage with m1'])
        print('-----------------Ours-----------------')
        print(metrics_test)

    ours_acc.append(ours_acc_trial)
    ours_hidden.append(ours_hidden_trial)
    ours_m1use.append(ours_m1use_trial)

ours_acc = np.array(ours_acc)
ours_hidden = np.array(ours_hidden)
ours_m1use = np.array(ours_m1use)

In [ ]:
ours_acc = np.array(ours_acc)
ours_hidden = np.array(ours_hidden)
ours_m1use = np.array(ours_m1use)

In [ ]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
alpha_overlap = 0.8


ns = cond_acc_targets

colors = plt.cm.viridis(np.linspace(0, 1, 7))
# Define a list of 7 markers
markers = ['o', 's', '^', 'v', 'D', 'p', 'X']


for i in range(len(m1_use_targets)):
    if m1_use_targets[i] == 0:
        label = f'CDHF with R={m1_use_targets[i]} ($m_2$ only)'
    elif m1_use_targets[i] == 1:
        label = f'CDHF with R={m1_use_targets[i]} ($m_1$ only)'
    else:
        label = f'CDHF with R={m1_use_targets[i]}'
    

    plt.errorbar(ours_acc[i]*100,  ours_hidden[i]*100,  marker = markers[i],  label=label, alpha =alpha_overlap, color = colors[i])




ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4.2
plt.grid()
plt.legend(fontsize='xx-large')
plt.legend()
plt.ylabel(' Suggestions Hidden (\%)', fontsize='xx-large')
plt.xlabel('Accuracy When Hiding Suggestions (TNR)', fontsize='xx-large')

# make y axis markers in increments of 0.05
plt.yticks(np.arange(0, 101, 10))
plt.xticks(np.arange(78, 102, 2))



plt.savefig("plot_cdhf"+ date_now+".pdf", dpi = 1000, bbox_inches='tight')
plt.show()